In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, naive_bayes, svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
train_url = '/home/thinker/NLP/NLP/FromScratch/error detection/data/data/train_data.csv'
test_url = '/home/thinker/NLP/NLP/FromScratch/error detection/data/data/test_data.csv'
df_train = pd.read_csv(train_url)
df_test = pd.read_csv(test_url)
stop_words_df = pd.read_excel('/home/thinker/NLP/NLP/FromScratch/error detection/data/stopwords_bangla.xlsx',index_col=False)

In [2]:
STOPWORDS = set([word.strip() for word in stop_words_df['words']])

In [3]:
import re
def preprocess(x):
    html_pattern = re.compile('<.*?>')
    x = html_pattern.sub(r'', x)
    x = " ".join([word for word in str(x).split() if word not in STOPWORDS])
    return x
df_train['Comment'] = df_train['Comment'].apply(lambda x: preprocess(x))
df_test['Comment'] = df_test['Comment'].apply(lambda x:preprocess(x))

In [4]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(df_train.Error)
Test_Y = Encoder.fit_transform(df_test.Error)

In [5]:
df_all  = pd.concat([df_train, df_test], ignore_index=True)
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df_all['Comment'])
Train_X_Tfidf = Tfidf_vect.transform(df_train['Comment'])
Test_X_Tfidf = Tfidf_vect.transform(df_test['Comment'])

In [6]:
model = RandomForestClassifier(
    n_estimators=100,    # Number of decision trees in the forest
    max_depth=None,       # Maximum depth of individual trees
    min_samples_split=2,  # Minimum number of samples required to split an internal node
    min_samples_leaf=1,   # Minimum number of samples required to be at a leaf node
    random_state=42
)

In [7]:
model.fit(Train_X_Tfidf, Train_Y)

RandomForestClassifier(random_state=42)

In [8]:
y_pred = model.predict(Test_X_Tfidf)

In [9]:
print(metrics.classification_report(Test_Y, y_pred,digits = 4))

              precision    recall  f1-score   support

           0     0.6427    0.8046    0.7146      1167
           1     0.5855    0.3815    0.4620       844

    accuracy                         0.6271      2011
   macro avg     0.6141    0.5931    0.5883      2011
weighted avg     0.6187    0.6271    0.6086      2011

